# TASK 6 

# Recognizing Face and Doing Multiple Tasks After It

# Step 1 - Creating Training DataSet For ML Model

In [4]:
import cv2
import numpy as np

#Loading HaarCascade face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#Defining Function
def face_extractor(img):
    #Function detects faces and return the cropped faces
    #If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    # converting RGB colors of image to B&W or GRAY
    faces=face_classifier.detectMultiScale(gray, 1.3, 5)  
    # used for detecting face , here 1.3 is scaling factor and 5 is no. of nearest neighbours
    
    if faces is ():
        return None
    
    # crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h , x:x+w]
        
    return cropped_face

# Initializing webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samles of your face from webcam input
while True:
    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize( face_extractor(frame), (200 , 200) )
        face = cv2.cvtColor( face, cv2.COLOR_BGR2GRAY )
        
        #save file in specified directory with unique name
        file_name_path = r'C:\Users\ABHISHEK VERMA\Desktop\cv-sp\Face-Recognition--main\Face-Recognition--main\abhishek\imgesdatset-1/' + str(count) + '.jpg'
        cv2.imwrite( file_name_path, face)
        
        #put count on images and display live count
        cv2.putText( face , str(count), (50,50), cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow( 'Face cropper', face )
        
    else:
        print('Face Not Found')
        pass
    if cv2.waitKey(1) == 13 or count == 200:  # 13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()
print("---- SAMPLE IMAGES COLLECTED ----")
print("---- READY FOR TRAINING ----")

<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-4-7134f2cf75b3>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Face Not Found
Face Not Found
Face Not Found
Face Not Found
Face Not Found
Face Not Found
Face Not Found
Face Not Found
Face Not Found
Face Not Found
Face Not Found
Face Not Found
Face Not Found
---- SAMPLE IMAGES COLLECTED ----
---- READY FOR TRAINING ----


# Step 2 - Train Model

In [6]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile , join


# Get the trainig data we made previously
data_path = r'C:\Users\ABHISHEK VERMA\Desktop\cv-sp\Face-Recognition--main\Face-Recognition--main\abhishek\imgesdatset-1/'
onlyfiles = [f for f in listdir(data_path) if isfile ( join ( data_path, f ) ) ]

# Create array for training Data and labels
Training_Data , Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()
recognizer  = cv2.face.LBPHFaceRecognizer_create()
# Let's train our model 
recognizer.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully




# Step 3 - Run Our Facial Recognition

In [12]:
import cv2
import numpy as np
import os
import time
import pywhatkit
import subprocess as sp
import webbrowser as wb
import smtplib

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = recognizer.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 1, (255,120,150), 2)
        if confidence > 85:
            cv2.putText(image, "HY Abhishek Verma ", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            #This will launch instance in aws with name production
            a = sp.getstatusoutput("aws ec2 run-instances --image-id ami-0ad704c126371a549 --subnet-id subnet-59f8f131 --instance-type t2.micro  --count 1  --security-group-ids sg-d3da70b4 --key-name mykeytoaws --tag-specifications ResourceType=instance,Tags=[{Key=Name,Value=Production}] ")
            print(a[1])
            text1 ='"--------------New Instance Launched successfully----------------"'
            print(text1)
            wb.open("https://ap-south-1.console.aws.amazon.com/ec2/v2/home?region=ap-south-1#Instances:")   
            #This will launch external ebs  volume in aws with name external
            b = sp.getstatusoutput("aws ec2 create-volume --availability-zone ap-south-1a --volume-type gp2 --size 5 --tag-specifications ResourceType=volume,Tags=[{Key=Name,Value=ExternalVolume}]")
            print(b[1])
            text2 ='--------------New voume created successfully----------------'
            print(text2)
            wb.open("https://ap-south-1.console.aws.amazon.com/ec2/v2/home?region=ap-south-1#Volumes:sort=desc:createTime")            
            volume_id = input("enter volume id:")
            instance_id =input("enter instance id:")
            #This will attach external volumewith aws instance.
            c=sp.getstatusoutput("aws ec2 attach-volume --volume-id {} --instance-id {} --device /dev/sdf".format(volume_id,instance_id))
            print(c[1])
            text3 ="--------------New voume Added to Instance successfully----------------"
            print(text3)
            wb.open("https://ap-south-1.console.aws.amazon.com/ec2/v2/home?region=ap-south-1#Instances:")       
            break    
        else:
            cv2.putText(image, " Stanger ", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            cv2.waitKey() == 13

            # Python code to illustrate Sending mail from 
            # your Gmail account 
            

            # creates SMTP session
            s = smtplib.SMTP_SSL('smtp.gmail.com', 465)

            # start TLS for security
            #s.starttls()

            # Authentication
            s.login("vermaabhishek.av109@gmail.com", "Google#$9")

            # message to be sent
            message = "Detected Stanger face From Python"

            # sending the mail
            s.sendmail("vermaabhishek.av109@gmail.com", "abhishekverma.av109@gmail.com", message)

            # terminating the session
            s.quit()
            #pywhatkit.send_mail("vermaabhishek.av109@gmail.com", "Google#$9", "face recognition", "hello this is  Abhishek Verma", "abhishekverma.av109@gmail.com")
            #pywhatkit.sendwhatmsg_instantly(phone_no = "+918xxxxxxxxx", message = "Hi, i sent you this message by Face Recognition ")
            
            break

    except Exception as e:
        print(e)
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     

<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-12-8fa816dc257d>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


OpenCV(4.5.2) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

OpenCV(4.5.2) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

OpenCV(4.5.2) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

OpenCV(4.5.2) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

OpenCV(4.5.2) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

OpenCV(4.5.2) :-1: error: (-5:Bad a